In [1]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
!pip install timm


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import io

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class ParquetDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_parquet(file_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_data = self.data.iloc[idx]['image']
        label = self.data.iloc[idx]['label']
        if isinstance(image_data, dict):
            if "bytes" in image_data:
                image = Image.open(io.BytesIO(image_data["bytes"]))
            else:
                raise ValueError("未找到包含图像数据的键 'bytes'")
        else:
            raise ValueError("图像数据的格式不符合预期")

        image_tensor = transform(image)

        return image_tensor, label

file_paths = [
    'CRC_VAL_HE_7K-00000-of-00003-cce1097526b69125.parquet'
]

datasets = [ParquetDataset(file_path) for file_path in file_paths]

full_dataset = torch.utils.data.ConcatDataset(datasets)

batch_size = 32
data_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

for images, labels in data_loader:
    print(f'Image batch shape: {images.shape}')
    print(f'Label batch shape: {labels.shape}')
    break

Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])


In [8]:
import torch
import timm
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_small_patch16_224_dino', pretrained=True)
model.to(device)
model.eval()


/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [14]:
from PIL import Image

def extract_features(backbone, dataloader, device):
    features, labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            inputs, batch_labels = batch

            inputs = inputs.to(device)
            batch_labels = batch_labels.to(device)

            outputs = backbone.forward_features(inputs)

            features.extend(outputs.cpu().numpy())
            labels.extend(batch_labels.cpu().numpy())

    return features, labels

train_features, train_labels = extract_features(model, data_loader, device)

print(f"Extracted feature shape: {np.array(train_features).shape}")
print(f"Extracted labels shape: {np.array(train_labels).shape}")


Extracted feature shape: (2394, 197, 384)
Extracted labels shape: (2394,)


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [26]:

train_features_np = np.array(train_features)

n_samples = min(len(train_features_np), len(train_labels))
train_features_np = train_features_np[:n_samples]
train_labels_np = np.array(train_labels[:n_samples])

train_features_flatten = train_features_np.reshape(n_samples, -1)

input_dim = train_features_flatten.shape[1]
num_classes = np.max(train_labels_np) + 1
print(f"Number of classes (num_classes): {num_classes}")




Number of classes (num_classes): 8


In [23]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [27]:

class FullyConnectedModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FullyConnectedModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.fc(x)

train_features_np, val_features_np, train_labels_np, val_labels_np = train_test_split(train_features_flatten, train_labels_np, test_size=0.1, random_state=42)

train_features_tensor = torch.tensor(train_features_np, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels_np, dtype=torch.long)
val_features_tensor = torch.tensor(val_features_np, dtype=torch.float32)
val_labels_tensor = torch.tensor(val_labels_np, dtype=torch.long)

train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_features_tensor, val_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = train_features_np.shape[1]
# num_classes = len(np.unique(train_labels_np))+1
model = FullyConnectedModel(input_dim=input_dim, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("训练完成！")

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

y_pred = np.array(all_preds)
y_true = np.array(all_labels)

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


Epoch [1/10], Loss: 1.7234
Epoch [2/10], Loss: 1.7165
Epoch [3/10], Loss: 1.7155
Epoch [4/10], Loss: 1.7165
Epoch [5/10], Loss: 1.7155
Epoch [6/10], Loss: 1.7175
Epoch [7/10], Loss: 1.7155
Epoch [8/10], Loss: 1.7165
Epoch [9/10], Loss: 1.7175
Epoch [10/10], Loss: 1.7155
训练完成！
Accuracy: 0.5875
F1 Score: 0.4348
Precision: 0.3452
Recall: 0.5875
Confusion Matrix:
[[141   0   0   0]
 [ 30   0   0   0]
 [ 28   0   0   0]
 [ 41   0   0   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      1.00      0.74       141
           2       0.00      0.00      0.00        30
           3       0.00      0.00      0.00        28
           7       0.00      0.00      0.00        41

    accuracy                           0.59       240
   macro avg       0.15      0.25      0.19       240
weighted avg       0.35      0.59      0.43       240



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [32]:

train_features_np = np.array(train_features)

n_samples = min(len(train_features_np), len(train_labels))
train_features_np = train_features_np[:n_samples]
train_labels_np = np.array(train_labels[:n_samples])

train_features_flatten1 = train_features_np.mean(axis=1)

input_dim = train_features_flatten.shape[1]
num_classes = np.max(train_labels_np) + 1
print(f"Number of classes (num_classes): {num_classes}")



Number of classes (num_classes): 8


In [33]:

class FullyConnectedModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FullyConnectedModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.fc(x)

train_features_np, val_features_np, train_labels_np, val_labels_np = train_test_split(train_features_flatten1, train_labels_np, test_size=0.1, random_state=42)

train_features_tensor = torch.tensor(train_features_np, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels_np, dtype=torch.long)
val_features_tensor = torch.tensor(val_features_np, dtype=torch.float32)
val_labels_tensor = torch.tensor(val_labels_np, dtype=torch.long)

train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_features_tensor, val_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = train_features_np.shape[1]
# num_classes = len(np.unique(train_labels_np))+1
model = FullyConnectedModel(input_dim=input_dim, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("训练完成！")

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

y_pred = np.array(all_preds)
y_true = np.array(all_labels)

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


Epoch [1/10], Loss: 1.3491
Epoch [2/10], Loss: 1.2774
Epoch [3/10], Loss: 1.2758
Epoch [4/10], Loss: 1.2753
Epoch [5/10], Loss: 1.2749
Epoch [6/10], Loss: 1.2746
Epoch [7/10], Loss: 1.2744
Epoch [8/10], Loss: 1.2742
Epoch [9/10], Loss: 1.2748
Epoch [10/10], Loss: 1.2744
训练完成！
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
Confusion Matrix:
[[141   0   0   0]
 [  0  30   0   0]
 [  0   0  28   0]
 [  0   0   0  41]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       141
           2       1.00      1.00      1.00        30
           3       1.00      1.00      1.00        28
           7       1.00      1.00      1.00        41

    accuracy                           1.00       240
   macro avg       1.00      1.00      1.00       240
weighted avg       1.00      1.00      1.00       240

